# Problem 103

Let S(A) represent the sum of elements in set A of size n. We shall call it a special sum set if for any two non-empty disjoint subsets, B and C, the following properties are true:

- S(B) ≠ S(C); that is, sums of subsets cannot be equal.
- If B contains more elements than C then S(B) > S(C).

If S(A) is minimised for a given n, we shall call it an optimum special sum set. The first five optimum special sum sets are given below.

```
n = 1: {1}
n = 2: {1, 2}
n = 3: {2, 3, 4}
n = 4: {3, 5, 6, 7}
n = 5: {6, 9, 11, 12, 13}
```

It seems that for a given optimum set, `A = {a1, a2, ... , an}`, the next optimum set is of the form `B = {b, a1+b, a2+b, ... ,an+b}`, where b is the "middle" element on the previous row.

By applying this "rule" we would expect the optimum set for n = 6 to be `A = {11, 17, 20, 22, 23, 24}`, with `S(A) = 117`. However, this is not the optimum set, as we have merely applied an algorithm to provide a near optimum set. The optimum set for n = 6 is `A = {11, 18, 19, 20, 22, 25}`, with S(A) = 115 and corresponding set string: 111819202225.

Given that A is an optimum special sum set for n = 7, find its set string.

In [1]:
from dataclasses import dataclass
from typing import Dict, Set, List


@dataclass
class Solution:
    values: List[int]
    sets: List[Set[int]]
    min_sum_by_size: Dict[int, int]
    max_sum_by_size: Dict[int, int]
    sums: Set[int]
        
    @classmethod
    def from_initial_value(cls, v):
        solution = Solution(
            values=[],
            sets=list(),
            sums=set(),
            min_sum_by_size=dict(),
            max_sum_by_size=dict()
        )
        solution.values.append(v)
        solution.add_set({v})
        return solution
    
    def add_set(self, s) -> bool:
        set_size, set_sum = len(s), sum(s)
        
        if set_sum in self.sums:
            return False
        
        if set_sum < self.max_sum_by_size.get(set_size - 1, 0):
            return False
        
        if set_sum > self.min_sum_by_size.get(set_size + 1, 1000000):
            return False
        
        self.sets.append(s)
        self.min_sum_by_size[set_size] = min(self.min_sum_by_size.get(set_size, 1000000), set_sum)
        self.max_sum_by_size[set_size] = max(self.max_sum_by_size.get(set_size, 0), set_sum)
        self.sums.add(set_sum)
        return True
    
    def combine_and_add(self, n) -> bool:
        for s in list(self.sets):
            sc = copy(s)
            sc.add(n)
            has_added = self.add_set(sc)
            if not has_added:
                return False
        return True
    
    def do_step(self, n):
        v1 = self.combine_and_add(n)
        v2 = self.add_set({n})
        return v1, v2

In [2]:
from copy import deepcopy, copy

BEST_SUM = 1000000
BEST_SET = None
TARGET_SIZE = 7

def execute(s=None):
    global BEST_SUM, BEST_SET
    
    if not s:
        i = 1
        best_sum_achievable = sum(range(i, i+TARGET_SIZE))
        while best_sum_achievable < BEST_SUM:
            solution = Solution.from_initial_value(i)
            execute(solution)
            i += 1
            best_sum_achievable = sum(range(i, i+TARGET_SIZE))
        print(f"Stopped execution at start point = {i}") 

    elif len(s.values) == TARGET_SIZE:
        the_sum = sum(s.values)
        if the_sum < BEST_SUM:
            BEST_SUM = the_sum
            BEST_SET = copy(s.values)
            print(f"\nFOUND\nFound best set {BEST_SET} with best sum {BEST_SUM}\nFOUND\n")
            
    else:
        n = s.values[-1] + 1
        
        best_sum_achievable = sum(s.values) + sum(range(n, n + TARGET_SIZE - len(s.values)))
        while best_sum_achievable < BEST_SUM and n < 50:
            if n > s.min_sum_by_size.get(2, 1000000):
                break
                
            ss = deepcopy(s)
            
            
            has_added = ss.combine_and_add(n)
            if not has_added:
                #print(f"Skip execution at {s.values} with next_to_add={n} because sets insertion was not successful")
                n += 1
                best_sum_achievable = sum(s.values) + sum(range(n, n + TARGET_SIZE - len(s.values)))
                continue
            
            has_added = ss.add_set({n})
            if not has_added:
                #print(f"Skip execution at {s.values} with next_to_add={n} because insertion was not successful")
                n += 1
                best_sum_achievable = sum(s.values) + sum(range(n, n + TARGET_SIZE - len(s.values)))
                continue
                    
            ss.values.append(n)
            
            execute(ss)
            
            n += 1
            best_sum_achievable = sum(s.values) + sum(range(n, n + TARGET_SIZE - len(s.values)))
            
        #print(f"Stopped execution at {s.values} with next_to_add={n}")

In [3]:
execute()


FOUND
Found best set [20, 31, 38, 39, 40, 42, 45] with best sum 255
FOUND

Stopped execution at start point = 34


In [4]:
BEST_SET, BEST_SUM

([20, 31, 38, 39, 40, 42, 45], 255)